In [ ]:
# default_exp LNScrape

# LNScrape

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
import time
# import pickle
from selenium import webdriver
from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager


In [ ]:
### SET UP ###

BOOLEAN_QUERY = "(climat* W/5 (chang* OR catastroph* OR disaster* OR transform* OR adjust* OR trend* OR world* OR \
    earth* OR warm* OR heat* OR cool* OR variab*)) OR ((greenhouse* W/3 effect*) OR ((global* OR earth* \
    OR world* OR international* OR hemisphere*) W/5 (warm* OR heat* OR cool* OR chill*)) OR ((temperature*W/5 \
    (global* OR earth* OR world* OR international* OR hemisphere*) W/8 (increas* OR rising* OR rise* OR decreas*))"
SOURCES = ["USA Today"]
START_PAGE = 1
ACCOUNT = "username"
PW = "password"


In [ ]:
import time
# import pickle
from selenium import webdriver
from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

from webdriver_manager.chrome import ChromeDriverManager


# ["The New York Times",
#            "Herald Sun/Sunday Herald Sun (Melbourne, Australia)",
#            "The Globe and Mail(Canada)",
#            "National Post (f/k/a The Financial Post)(Canada)",
#            "People's Daily Online - English"]


driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.implicitly_wait(10)
# driver.set_page_load_timeout(10)

driver.get("https://signin.lexisnexis.com/lnaccess/app/signin?back=https%3A%2F%2Fadvance.lexis.com%3A443%2FBisNexisPowerSearch&aci=la")
# driver.get("https://advance.lexis.com/search/?pdmfid=1519360&crid=0cea597d-99ec-4730-9875-df93af284ca7&pdsearchterms=(climat*+W%2F5+(chang*+OR+catastroph*+OR+disaster*+OR+transform*+OR+adjust*+OR+trend*+OR+world*+OR+earth*+OR+warm*+OR+heat*+OR+cool*+OR+variab*))+OR+((greenhouse*+W%2F3+effect*)+OR+((global*+OR+earth*+OR+world*+OR+international*+OR+hemisphere*)+W%2F5+(warm*+OR+heat*+OR+cool*+OR+chill*))+OR+((temperature*W%2F5+(global*+OR+earth*+OR+world*+OR+international*+OR+hemisphere*)+W%2F8+(increas*+OR+rising*+OR+rise*+OR+decreas*))&pdstartin=hlct%3A1%3A1&pdcaseshlctselectedbyuser=false&pdtypeofsearch=searchboxclick&pdsearchtype=SearchBox&pdoriginatingpage=bisnexishome&pdqttype=or&pdsf=MTA2NzgzOQ~%5Enews~%5EThe+Times+(London)*~%5Efalse~%5EThe%2520Times%2520(London)&pdquerytemplateid=&indexsearch=false&ecomp=_x7hkkk&earg=pdsf&prid=b8606906-f40f-45da-b1e8-1c789b605eb2")
wait = WebDriverWait(driver, 20)


def find_and_click(locator, session=driver, by=By.CSS_SELECTOR):
    done = False
    while not done:
        try:
            element = session.find_element(by, locator)
            actions = ActionChains(session)
            actions.move_to_element(element).perform()
            session.execute_script("arguments[0].click();", element)
            done = True
        except StaleElementReferenceException:
            pass
        
### SET UP ###
if "Sign In" in driver.title:
    driver.find_element(By.ID, "userid").click()
    driver.find_element(By.ID, "userid").send_keys(ACCOUNT)
    driver.find_element(By.ID, "password").send_keys(PW)
    driver.find_element(By.ID, "rememberMe").click()
    driver.find_element(By.ID, "signInSbmtBtn").click()

assert "Power Search" in driver.title
# driver.fullscreen_window()

correct_results = False
repeat = False
while not correct_results:
    # if repeat:
    #     wait.until(expected_conditions.presence_of_element_located(
    #         (By.ID, ".btn"))).click()
    # else:
    driver.find_element(By.ID, "searchTerms").click()
    driver.execute_script("window.scrollTo(0,105.19999694824219)")
    driver.find_element(By.ID, "searchTerms").send_keys(BOOLEAN_QUERY)

    wait.until(expected_conditions.presence_of_element_located(
        (By.ID, "nexis-index-filters-tab"))).click()
    wait.until(expected_conditions.presence_of_element_located(
        (By.ID, "bisPowerSearchNewsSection"))).click()
    wait.until(expected_conditions.presence_of_element_located(
        (By.ID, "Newspapers"))).click()
    wait.until(expected_conditions.presence_of_element_located(
        (By.ID, "English"))).click()

    driver.find_element(By.ID, "nexis-sources-filters-tab").click()
    wait.until(expected_conditions.presence_of_element_located(
        (By.ID, "sourcesSearchTerms"))).click()

    for SOURCE in SOURCES:
        try:
            driver.find_element(By.ID, "sourcesSearchTerms").clear()
            for s in SOURCE:
                driver.find_element(By.ID, "sourcesSearchTerms").send_keys(s)
                time.sleep(0.01)

            driver.find_element(By.ID, "sourcesSearchTerms").send_keys(" ")
            driver.find_element(
                By.ID, "sourcesSearchTerms").send_keys(Keys.BACKSPACE)
            wait.until(expected_conditions.text_to_be_present_in_element(
                (By.XPATH, '//*[@id="source"]/ul/li[1]'), SOURCE))

            # wait.until(expected_conditions.staleness_of(
            #     (By.CSS_SELECTOR, "#source > ul > li:nth-child(1) > div > a"))).click()
            find_and_click("#source > ul > li:nth-child(1) > div > a", driver)
            find_and_click("body", driver)
            time.sleep(0.5)
            driver.find_element(By.ID, "sourcesSearchTerms").clear()
        except Exception as e:
            pass

    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, ".btn").click()

    results_list = "#content > div.results-list-container > form"
    element = wait.until(expected_conditions.presence_of_element_located(
        (By.CSS_SELECTOR, results_list)))
    # wait.until(expected_conditions.staleness_of(element))

    if "0 results" in driver.title:
        driver.get(
            "https://signin.lexisnexis.com/lnaccess/app/signin?back=https%3A%2F%2Fadvance.lexis.com%3A443%2FBisNexisPowerSearch&aci=la")
        repeat = True
        continue
    else:
        correct_results = True


max_page = driver.find_element(
    By.CSS_SELECTOR, "#content > div.results-list-container > form > div.wrapper > nav > ol > li:nth-child(6)")
max_page_num = int(max_page.text)




====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\t-zha\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [ ]:
def select_results_hitcount(session=driver, threshold=10):
    selected_count = 0
    results = session.find_elements_by_css_selector("[class^='row_sr']")
    for result in results:
        try:
            hits = result.find_element_by_xpath(
                '//*[@id="bisnexis-flex"]/div[2]/ul/li[3]/div').text
            num_hits = int(hits.replace(" hits", ""))

            if num_hits >= threshold:
                element = result.find_element_by_css_selector(
                    "[class^='tools-toggle skip-target']")
                actions = ActionChains(session)
                actions.move_to_element(element).perform()
                session.execute_script("arguments[0].click();", element)
                selected_count += 1

        except Exception as e:
            continue

    return selected_count


In [ ]:
driver.find_element_by_css_selector('')

In [ ]:
x = driver.find_elements_by_css_selector("[class^='row_sr']")[-1]


In [ ]:
x.find_element_by_xpath(
    '//*[@id="bisnexis-flex"]/div[2]/ul/li[3]/div').text


'19 hits'

In [ ]:
def download():
    time.sleep(2)
    WebDriverWait(driver, 180, 1).until(expected_conditions.element_to_be_clickable(
        (By.CSS_SELECTOR, ".has_tooltip:nth-child(1) > .la-Download"))).click()
    WebDriverWait(driver, 180, 1).until(expected_conditions.element_to_be_clickable(
        (By.CSS_SELECTOR, ".button-group > .primary"))).click()

    print('starting download')
    try:
        WebDriverWait(driver, 180, 3).until(
            expected_conditions.visibility_of_element_located((By.CSS_SELECTOR, "#delivery-jobs > li > img")))
        WebDriverWait(driver, 180, 3).until(
            expected_conditions.visibility_of_element_located((By.CSS_SELECTOR, "#delivery-popin > img")))
        print('waiting...')
        time.sleep(5)
        WebDriverWait(driver, 180, 3).until(
            expected_conditions.invisibility_of_element_located((By.CSS_SELECTOR, "#delivery-popin > img")))
        WebDriverWait(driver, 180, 3).until(
            expected_conditions.invisibility_of_element_located((By.CSS_SELECTOR, "#delivery-jobs > li > img")))
    except:
        pass
    print("Downloaded!")


In [ ]:
def click_next_page(driver, page_num):
    try:
        element = WebDriverWait(driver, 20).until(expected_conditions.element_to_be_clickable(
            (By.LINK_TEXT, str(page_num))))
        # driver.find_element(By.LINK_TEXT, str(page))
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.click()
    except Exception as e:
        pass


In [ ]:
import random
# START_PAGE = 4720

on_page = driver.find_element(
    By.CSS_SELECTOR, "#content > div.results-list-container > form > div.wrapper > nav > ol > li.current > span").text

for page in range(int(on_page), max_page_num+1):
    time.sleep(random.randrange(0, 6))
    if page < START_PAGE:
        click_next_page(driver, page+2)
        continue
    elif page == 1:
        # Select page 1 results
        total_select = select_results_hitcount(driver, threshold=5)
    else:
        click_next_page(driver, page)

        # Wait for next page to load
        try:
            wait.until(expected_conditions.invisibility_of_element_located(
                (By.CSS_SELECTOR, "#loadbox > div > img")))
        except:
            print("Timed out")

        # Selecting just those with n hits
        selected = select_results_hitcount(driver, threshold=5)

        try:
            total_select = wait.until(expected_conditions.visibility_of_element_located((By.CSS_SELECTOR,
                                                                                         "#results-list-delivery-toolbar > div > ul:nth-child(1) > \
                                                                                        li.selected > div > button > span:nth-child(1)"))).text
        except Exception as e:
            continue

        if int(total_select) >= 450:
            download()
            total_select = 0

            # Clear selected
            time.sleep(1)
            find_and_click(
                "#results-list-delivery-toolbar > div > ul:nth-child(1) > li.selected > div > button")
            time.sleep(0.5)
            wait.until(expected_conditions.element_to_be_clickable(
                (By.CSS_SELECTOR, "#viewtray-dropdown > div > div.dropdown-header > div > button:nth-child(2)"))).click()
            time.sleep(0.5)

            wait.until(expected_conditions.element_to_be_clickable(
                (By.CSS_SELECTOR, "body > aside > footer > div > button.button.primary"))).click()
            time.sleep(1)
            driver.refresh()
            WebDriverWait(driver, 20).until(
                expected_conditions.presence_of_element_located((By.LINK_TEXT, str(page+1))))


download()


In [ ]:
import selenium

selenium.__version__

'3.141.0'

In [ ]:
#export
import time
import random
# import pickle
from selenium import webdriver
from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager

class LNScrape():
    
    def __init__(self,
                 query: str,
                 sources: list,
                 username: str,
                 password: str):
        self.query = query
        self.sources = sources
        self.username = username
        self.password = password
        self.max_page = 0
        
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        self.wait = WebDriverWait(self.driver, 20)
        
        self.set_up()
        
    def find_and_click(self, locator, by=By.CSS_SELECTOR):
        """utillitiy function for finding a specific lcoator, scrolling into view and clicking it

        Args:
            locator (str): The locator of the element
            by (selenium.webdriver.common.by, optional): Selenium By type. Defaults to By.CSS_SELECTOR.
        """
        done = False
        while not done:
            try:
                element = self.driver.find_element(by, locator)
                actions = ActionChains(self.driver)
                actions.move_to_element(element).perform()
                self.driver.execute_script("arguments[0].click();", element)
                done = True
            except StaleElementReferenceException:
                pass
            
    def select_results_hitcount(self, threshold=10):
        """Select only the results that have at least the threshold number of keyword/boolean hits
           and returns the count of articles selected.

        Args:
            threshold (int, optional): The keyword hits threshold. Defaults to 10.

        Returns:
            int: count of articles selected
        """
        selected_count = 0
        results = self.driver.find_elements_by_css_selector("[class^='row_sr']")
        for result in results:
            try:
                hits = result.find_element_by_xpath(
                    '//*[@id="bisnexis-flex"]/div[2]/ul/li[3]/div').text
                num_hits = int(hits.replace(" hits", ""))

                if num_hits >= threshold:
                    element = result.find_element_by_css_selector(
                        "[class^='tools-toggle skip-target']")
                    actions = ActionChains(self.driver)
                    actions.move_to_element(element).perform()
                    self.driver.execute_script("arguments[0].click();", element)
                    selected_count += 1

            except Exception as e:
                print(e)
                continue

        return selected_count
    
    def click_next_page(self, page_num):
        try:
            element = WebDriverWait(self.driver, 20).until(expected_conditions.element_to_be_clickable(
                (By.LINK_TEXT, str(page_num))))
            # driver.find_element(By.LINK_TEXT, str(page))
            self.driver.execute_script("arguments[0].scrollIntoView(true);", element)
            element.click()
        except Exception as e:
            pass
    
    
    def download(self):
        time.sleep(2)
        WebDriverWait(self.driver, 180, 1).until(expected_conditions.element_to_be_clickable(
            (By.CSS_SELECTOR, ".has_tooltip:nth-child(1) > .la-Download"))).click()
        WebDriverWait(self.driver, 180, 1).until(expected_conditions.element_to_be_clickable(
            (By.CSS_SELECTOR, ".button-group > .primary"))).click()

        print('starting download')
        try:
            WebDriverWait(self.driver, 180, 3).until(
                expected_conditions.visibility_of_element_located((By.CSS_SELECTOR, "#delivery-jobs > li > img")))
            WebDriverWait(self.driver, 180, 3).until(
                expected_conditions.visibility_of_element_located((By.CSS_SELECTOR, "#delivery-popin > img")))
            print('waiting...')
            time.sleep(5)
            WebDriverWait(self.driver, 180, 3).until(
                expected_conditions.invisibility_of_element_located((By.CSS_SELECTOR, "#delivery-popin > img")))
            WebDriverWait(self.driver, 180, 3).until(
                expected_conditions.invisibility_of_element_located((By.CSS_SELECTOR, "#delivery-jobs > li > img")))
        except:
            pass
        print("Downloaded!")


    def set_up(self):
        """Sets up the scraping session by logging on, entering the query string, and
           sources to search, then running the search. 
        
        """
        self.driver.get(
            "https://signin.lexisnexis.com/lnaccess/app/signin?back=https%3A%2F%2Fadvance.lexis.com%3A443%2FBisNexisPowerSearch&aci=la")

        ### SET UP ###
        if "Sign In" in self.driver.title:
            self.driver.find_element(By.ID, "userid").click()
            self.driver.find_element(By.ID, "userid").send_keys(self.username)
            self.driver.find_element(By.ID, "password").send_keys(self.password)
            self.driver.find_element(By.ID, "rememberMe").click()
            self.driver.find_element(By.ID, "signInSbmtBtn").click()

        assert "Power Search" in self.driver.title
        # self.driver.fullscreen_window()

        correct_results = False
        # repeat = False
        while not correct_results:
            self.driver.find_element(By.ID, "searchTerms").click()
            self.driver.execute_script("window.scrollTo(0,105.19999694824219)")
            self.driver.find_element(By.ID, "searchTerms").send_keys(self.query)

            self.wait.until(expected_conditions.presence_of_element_located(
                (By.ID, "nexis-index-filters-tab"))).click()
            self.wait.until(expected_conditions.presence_of_element_located(
                (By.ID, "bisPowerSearchNewsSection"))).click()
            self.wait.until(expected_conditions.presence_of_element_located(
                (By.ID, "Newspapers"))).click()
            self.wait.until(expected_conditions.presence_of_element_located(
                (By.ID, "English"))).click()

            self.driver.find_element(By.ID, "nexis-sources-filters-tab").click()
            self.wait.until(expected_conditions.presence_of_element_located(
                (By.ID, "sourcesSearchTerms"))).click()

            for SOURCE in self.sources:
                try:
                    self.driver.find_element(By.ID, "sourcesSearchTerms").clear()
                    for s in SOURCE:
                        self.driver.find_element(By.ID, "sourcesSearchTerms").send_keys(s)
                        time.sleep(0.01)

                    self.driver.find_element(By.ID, "sourcesSearchTerms").send_keys(" ")
                    self.driver.find_element(
                        By.ID, "sourcesSearchTerms").send_keys(Keys.BACKSPACE)
                    self.wait.until(expected_conditions.text_to_be_present_in_element(
                        (By.XPATH, '//*[@id="source"]/ul/li[1]'), SOURCE))

                    # self.wait.until(expected_conditions.staleness_of(
                    #     (By.CSS_SELECTOR, "#source > ul > li:nth-child(1) > div > a"))).click()
                    find_and_click("#source > ul > li:nth-child(1) > div > a", self.driver)
                    find_and_click("body", self.driver)
                    time.sleep(0.5)
                    self.driver.find_element(By.ID, "sourcesSearchTerms").clear()
                except Exception as e:
                    pass

            time.sleep(2)
            self.driver.find_element(By.CSS_SELECTOR, ".btn").click()

            results_list = "#content > div.results-list-container > form"
            element = self.wait.until(expected_conditions.presence_of_element_located(
                (By.CSS_SELECTOR, results_list)))
            # self.wait.until(expected_conditions.staleness_of(element))

            if "0 results" in self.driver.title:
                self.driver.get(
                    "https://signin.lexisnexis.com/lnaccess/app/signin?back=https%3A%2F%2Fadvance.lexis.com%3A443%2FBisNexisPowerSearch&aci=la")
                # repeat = True
                continue
            else:
                correct_results = True


        max_page = self.driver.find_element(
            By.CSS_SELECTOR, "#content > div.results-list-container > form > div.wrapper > nav > ol > li:nth-child(6)")
        self.max_page = int(max_page.text)
        
        
    def scrape(self, start_page=1, download_thresh=450):
        """This function takes an integer representing the first page to begin selecting articles from and runs until the
        last page of the search results. When the download_thresh representing the number of articles to download is reached, 
        the function will download the articles. You can also terminate this function earlier and calling the download function at any time. 
        

        Args:
            start_page (int, optional): The page from which to begin scraping from. This is useful for when the scraper stops unexpectedly
            at a certain page. Defaults to 1.
            download_thresh (int, optional): The number of articles to download at once. 500 is the max allowable by LN. Defaults to 450
        """
        on_page = self.driver.find_element(
            By.CSS_SELECTOR, "#content > div.results-list-container > form > div.wrapper > nav > ol > li.current > span").text
        print(on_page)
        for page in range(int(on_page), self.max_page+1):
            time.sleep(random.randrange(0, 6))
            if page < start_page:
                self.click_next_page(page+2)
                continue
            elif page == 1:
                # Select page 1 results
                total_select = self.select_results_hitcount(threshold=5)
            else:
                self.click_next_page(page)

                # Wait for next page to load
                try:
                    self.wait.until(expected_conditions.invisibility_of_element_located(
                        (By.CSS_SELECTOR, "#loadbox > div > img")))
                except:
                    print("Timed out")

                # Selecting just those with n hits
                selected = self.select_results_hitcount(threshold=5)

                try:
                    total_select = self.wait.until(expected_conditions.visibility_of_element_located((By.CSS_SELECTOR,
                                                                                                "#results-list-delivery-toolbar > div > ul:nth-child(1) > \
                                                                                                li.selected > div > button > span:nth-child(1)"))).text
                except Exception as e:
                    continue

                if int(total_select) >= download_thresh:
                    self.download()
                    total_select = 0

                    # Clear selected
                    time.sleep(1)
                    self.find_and_click(
                        "#results-list-delivery-toolbar > div > ul:nth-child(1) > li.selected > div > button")
                    time.sleep(0.5)
                    self.wait.until(expected_conditions.element_to_be_clickable(
                        (By.CSS_SELECTOR, "#viewtray-dropdown > div > div.dropdown-header > div > button:nth-child(2)"))).click()
                    time.sleep(0.5)

                    self.wait.until(expected_conditions.element_to_be_clickable(
                        (By.CSS_SELECTOR, "body > aside > footer > div > button.button.primary"))).click()
                    time.sleep(1)
                    self.driver.refresh()
                    WebDriverWait(self.driver, 20).until(
                        expected_conditions.presence_of_element_located((By.LINK_TEXT, str(page+1))))


        self.download()


In [ ]:
lnscrape = LNScrape(BOOLEAN_QUERY, SOURCES, ACCOUNT, PW)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\t-zha\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [ ]:
lnscrape.scrape()

In [ ]:
%cd "D:\Projects\LNScrape\lnscrape\"
%pwd


D:\Projects\LNScrape\lnscrape


'D:\\Projects\\LNScrape\\lnscrape'

In [ ]:
!nbdev_build_lib

Converted 00_LNScrape.ipynb.
Converted index.ipynb.
